<a href="https://colab.research.google.com/github/kaho0/Aaagh-more-math/blob/main/Module_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_curve,
)

plt.rcParams["figure.figsize"] = (7, 5)

#  Module 11.3 – Building a Simple Decision Tree (Synthetic Data)

This module introduces decision trees using a very small, easy-to-understand dataset.  
The goal is **intuition**, not accuracy.

---

##  Objectives

- Create a **tiny synthetic dataset**  
- Train a **Decision Tree Classifier**  
- Visualize the **learned tree**  
- Understand **how the tree makes decisions**

---

# 🔹 11.3.1 Create a Small Synthetic Dataset

For this example, we use a toy dataset with just two input features:

### **Features**
- **Weather**: `"Sunny"` or `"Rainy"`
- **Windy**: `0` = No, `1` = Yes  

### **Target**
- **Play**:  
  - `1` = Play  
  - `0` = Do not play  

We will manually create this dataset using a pandas DataFrame.

---

##  Code: Create the Dataset

```python
import pandas as pd

# Tiny synthetic dataset
data = {
    "Weather": ["Sunny", "Sunny", "Rainy", "Rainy", "Sunny", "Rainy"],
    "Windy":   [0,       1,       0,       1,       0,       1],
    "Play":    [1,       0,       1,       0,       1,       0]
}

df = pd.DataFrame(data)
df


# 🔹 11.3.2 Encode Categorical Feature and Prepare X, y

Machine learning models in sklearn expect numeric inputs.  
We will encode the categorical feature:

- **Sunny → 1**  
- **Rainy → 0**  

The feature **Windy** is already numeric (0/1), so we use it directly.  
The target variable **Play** is already 0/1.

---
#

In [3]:
data = {
    'Weather': ['Sunny', 'Rainy', 'Sunny', 'Sunny', 'Rainy', 'Rainy', 'Sunny', 'Rainy'],
    'Windy':   [0,        1,       0,        1,        0,        1,        0,       1],
    'Play':    [1,        0,       1,        1,        0,        0,        1,       0]
}
df_synthetic = pd.DataFrame(data)
df_synthetic


,Weather,Windy,Play
0,Sunny,0,1
1,Rainy,1,0
2,Sunny,0,1
3,Sunny,1,1
4,Rainy,0,0
5,Rainy,1,0
6,Sunny,0,1
7,Rainy,1,0


# 🔹 11.3.3 Train a Simple Decision Tree

Now we train a Decision Tree Classifier.

We set `max_depth=3` just to keep the tree small and easily interpretable.
For this introductory example, we fit the model on all available data, as the primary goal is to build intuition rather than perform rigorous evaluation.

In [4]:
df_synthetic['Weather_num']=df_synthetic['Weather'].map({'Sunny':1,'Rainy':0})
x_syn=df_synthetic[['Weather_num','Windy']]
y_syn=df_synthetic['Play']
display(x_syn)
display(y_syn)

,Weather_num,Windy
0,1,0
1,0,1
2,1,0
3,1,1
4,0,0
5,0,1
6,1,0
7,0,1


,Play
0,1
1,0
2,1
3,1
4,0
5,0
6,1
7,0
